In [35]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import IPython.display as ipd
import plotly.express as px
import librosa.display
import pandas as pd
import numpy as np
import librosa
import warnings
import glob
import IPython
import os
from tqdm import tqdm

#必要なライブラリの読み込み
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import os
from glob import glob
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from matplotlib import pyplot as plt
import pandas as pd
import cv2

warnings.simplefilter('ignore')

plt.style.use("ggplot")

In [36]:
exec_place = "local_machine"

In [37]:
##############################
# データセットのパスを設定する
##############################

#ローカル用
if exec_place == "local_machine":
    import win32com.client
    dir = "mydata.lnk"
    wshell = win32com.client.Dispatch("WScript.Shell") # <COMObject WScript.Shell>
    dir = wshell.CreateShortcut(dir).TargetPath
    
#Kaggle Notebook用
if exec_place == "kaggle_kernel":
    dir = "../input/birdsong-recognition/"
    
if not(os.path.exists(os.path.join(dir))):
    os.makedirs(dir)
    
print("dir:",dir)

dir: D:\kaggle_data\birdsong-recognition\mydata


In [38]:
#画像のサイズ指定
ScaleTo = 100
seed = 7
n_categories = 12

#トレーニングデータの読み込み
data_dir = os.path.join(dir,"train_melcepstrum")
path = os.path.join(data_dir,"*","*.pkl")
files = glob(path)

trainImg = []
trainLabel = []
j = 1
num = len(files)

#画像データをリストに格納
for f in files[:300]:
    print(str(j) + "/" + str(num) , end="\r")
    
    img = pd.read_pickle(f)
    img = img.values
    img = cv2.resize(img,(ScaleTo,ScaleTo))
    img = img[:,:,np.newaxis]
    
    #print(img)
    trainImg.append(img)
    trainLabel.append(f.split(os.path.sep)[-2])
    
    if j <= 0:
        plt.imshow(img[:,:,0])
        plt.show()
    
    j += 1


#ラベルをリストに格納
for dir in os.listdir(data_dir):
    if dir == ".DS_Store":
        continue
    dir1 = data_dir + "/" + dir
    label = dir

    for file in os.listdir(dir1):
        if file != "Thumbs.db":

            trainLabel.append(label)

In [39]:
# kerasに渡すためにnumpy配列に変換。
image_list = np.asarray(trainImg)
label_list = pd.DataFrame(trainLabel)


clearTrainImg = []
examples = [];getEx = True

for img in image_list:
    #ぼかしを入れてノイズを除去
    blurImg = cv2.GaussianBlur(img ,(5,5),0)
    #RGBからHSVに変換
    hsvImg = cv2.cvtColor(blurImg , cv2.COLOR_BGR2HSV)
    #マスクを作成
    lower_green = (25,40,50)
    upper_green = (75,255,255)
    mask = cv2.inRange(hsvImg , lower_green ,upper_green)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))
    mask = cv2.morphologyEx(mask , cv2.MORPH_CLOSE,kernel)

    #ブールマスクの作成
    bMask = mask > 0

    #マスクの適用
    #空のイメージの作成
    clear = np.zeros_like(img , np.uint8)
    #オリジナル画像にブールマスクを適用
    clear[bMask] = img[bMask]

    clearTrainImg.append(clear)


for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(clearTrainImg[i])


clearTrainImg = np.asarray(clearTrainImg)
clearTrainImg.shape
# クラスの形式を変換
le = LabelEncoder()
le = le.fit(label_list)
label_list = le.transform(label_list)
label_list
label_list = np_utils.to_categorical(label_list)
label_list

X_train, X_test, y_train, y_test = train_test_split(clearTrainImg, label_list, test_size=0.33, random_state=0)



from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=180,  # randomly rotate images in the range
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=True  # randomly flip images vertically
    )
datagen.fit(X_train)











#モデル作成・学習
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import tensorflow as tf
import keras
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D,Input
from keras.layers import Dense, Dropout, Flatten, Activation,GlobalAveragePooling2D,Input
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras import optimizers


# ResNet50のロード。FC層は不要なので include_top=False
input_tensor = Input(shape=(ScaleTo, ScaleTo, 3))
resnet50 = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

# FC層の作成
top_model = Sequential()
top_model.add(Flatten(input_shape=resnet50.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

# ResNet50とFC層を結合してモデルを作成
resnet50_model = Model(input=resnet50.input, output=top_model(resnet50.output))

#ResNet50の一部の重みを固定
for layer in resnet50_model.layers[:100]:
    layer.trainable = False

# 多クラス分類を指定
resnet50_model.compile(loss='categorical_crossentropy',
          optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
          metrics=['accuracy'])
resnet50_model.summary()


#学習の実行
hist = resnet50_model.fit_generator(datagen.flow(X_train, y_train, batch_size=75),
                        epochs=35, validation_data=(X_test, y_test),
                        steps_per_epoch=X_train.shape[0])



#モデルの評価
print(resnet50_model.evaluate(X_train, y_train)) #トレーニングの精度
print(resnet50_model.evaluate(X_test, y_test))  #テスト精度


#パラメータの保存
model.save_weights('../content/weights.h5')

ValueError: Found input variables with inconsistent numbers of samples: [300, 21674]

In [32]:
print(np.shape(img))

(3, 100, 100)
